In [1]:
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime as dt
from datetime import timedelta
import csv
import time
import pytz
from binance.client import Client
import pandas as pd
import os
from binance.spot import Spot

client = Spot()
client = Spot(api_key=os.getenv('Binance_API_KEY'), api_secret=os.getenv('Binance_secret_KEY'))


In [16]:
# Get the Account Information and print as a list and not a dictionary
account_info = client.account()
account_info = account_info['balances']
for item in account_info:
    if item['asset'] == 'USDT':
        print(item)

In [10]:
# Load the market data for all tickers with USDT
market_data = client.exchange_info()
market_data = market_data['symbols']
tickers = []    

In [ ]:
tickers = [item['symbol'] for item in market_data if item['symbol'].endswith('USDT')]
print(tickers)

In [ ]:
# Fetch the historical data for the tickers
def fetch_historical_data(ticker):
    data = client.klines(symbol=ticker, interval=Client.KLINE_INTERVAL_1HOUR)
    data = pd.DataFrame(data, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    data.set_index('timestamp', inplace=True)
    return data

# Fetch the historical data for all tickers
historical_data = {}
for ticker in tickers:
    historical_data[ticker] = fetch_historical_data(ticker)
    time.sleep(1)

In [17]:
# Try getting position size of each coin in my portfolio
def get_position_size(ticker):
    account_info = client.account()
    account_info = account_info['balances']
    for item in account_info:
        if item['asset'] == ticker:
            return float(item['free'])
    return 0

get_position_size('VTHO')

In [ ]:
# Try buying a new coin 
def buy(ticker, quantity):
    order = client.create_order(symbol=ticker, side='BUY', type='MARKET', quantity=quantity)
    return order              

In [21]:
# Try selling a coin
def sell(ticker, quantity):
    order = client.new_order(symbol=ticker, side='SELL', type='MARKET', quantity=quantity)

    return order

sell('VTHOUSDT', 0.08)

In [ ]:
# try getting exchange info
exchange_info = client.exchange_info()
exchange_info = exchange_info['symbols']
exchange_info = pd.DataFrame(exchange_info)
exchange_info = exchange_info[exchange_info['symbol'].isin(tickers)]
exchange_info = exchange_info[['symbol', 'baseAsset', 'quoteAsset', 'filters']]
exchange_info = exchange_info.reset_index(drop=True)
print(exchange_info[])

In [8]:
# quickly sell all coins in my portfolio
def sell_all():
    account_info = client.account()
    account_info = account_info['balances']
    for item in account_info:
        if float(item['free']) > 0:
            ticker = item['asset'] + 'USDT'
            quantity = float(item['free'])
            order = client.create_order(symbol=ticker, side='SELL', type='MARKET', quantity=quantity)
            print(order)
            time.sleep(1)